<table>
    <tr>
        <td><img src="./img/Macc.png" width="auto"/></td>
        <td>
            <table><tr>
            <h1 style="color:blue;text-align:center">Lógica para Ciencias de la Computación</h1></td>
            </tr></table>   
        <td>&nbsp;</td>
        <td>
            <table><tr>
            <tp><p style="font-size:150%;text-align:center">Taller</p></tp>
            <tp><p style="font-size:150%;text-align:center">Equivalencia lógica</p></tp>
            </tr></table>
        </td>
    </tr>
</table>

---

# Objetivos <a class="anchor" id="inicio"></a>

Una vez tenemos nuestra implementación de las fórmulas de la lógica proposicional como clases en Python, podemos comenzar a hacer pruebas interesantes. En este notebook vamos a determinar si dos fórmulas son equivalentes.

# Secciones

1. [Equivalencia lógica.](#eq)
2. [Sustituciones.](#sus)
3. [Froma normal conjuntiva.](#fnc)

# Equivalencia lógica <a class="anchor" id="eq"></a>

([Volver al inicio](#inicio))

Sean $A$, $B$, fórmulas. La equivalencia entre $A$ y $B$ (denotada por $A\equiv B$) se define de la siguiente manera:

$$
A\equiv B \mbox{ si, y solamente si, }A.valor(I)=B.valor(I)\mbox{ para toda interpretación }I
$$

Observe que ya habíamos usado la equivalencia anteriormente. Hicimos uso de ella para encontrar cuál era la prioridad de las operaciones booleanas en Python. En este caso revisamos las tablas de verdad de ambas variables para ver si las columnas eran todas iguales.

Vamos a operacionalizar esto en una función `chequear_equivalencia` en Python, la cual toma dos fórmulas (como árboles) como argumento. La función debe recorrer cada una de las interpretaciones posibles, y detener si no tienen el mismo valor. Si recorre todas sin detenerse, entonces son equivalentes.

Observe que tenemos acceso al archivo `Logica.py`, en el cual están implementadas las fórmulas y la función `inorder_to_tree`:

In [ ]:
'''
Librería con las clases y funciones
para lógica proposicional
'''

from itertools import product

class Formula :

    def __init__(self) :
        pass

    def __str__(self) :
        if type(self) == Letra:
            return self.letra
        elif type(self) == Negacion:
            return '-' + str(self.subf)
        elif type(self) == Binario:
            return "(" + str(self.left) + self.conectivo + str(self.right) + ")"

    def letras(self):
        if type(self) == Letra:
            return set(self.letra)
        elif type(self) == Negacion:
            return self.subf.letras()
        elif type(self) == Binario:
            return self.left.letras().union(self.right.letras())

    def subforms(self):
        if type(self) == Letra:
            return [str(self)]
        elif type(self) == Negacion:
            return list(set([str(self)] + self.subf.subforms()))
        elif type(self) == Binario:
            return list(set([str(self)] + self.left.subforms() + self.right.subforms()))

    def valor(self, I) :
        if type(self) == Letra:
            return I[self.letra]
        elif type(self) == Negacion:
            return not self.subf.valor(I)
        elif type(self) == Binario:
            if self.conectivo == 'Y':
                return self.left.valor(I) and self.right.valor(I)
            if self.conectivo == 'O':
                return self.left.valor(I) or self.right.valor(I)
            if self.conectivo == '>':
                return not self.left.valor(I) or self.right.valor(I)
            if self.conectivo == '=':
                return (self.left.valor(I) and self.right.valor(I)) or (not self.left.valor(I) and not self.right.valor(I))

class Letra(Formula) :
    def __init__ (self, letra:str) :
        self.letra = letra

class Negacion(Formula) :
    def __init__(self, subf:Formula) :
        self.subf = subf

class Binario(Formula) :
    def __init__(self, conectivo:str, left:Formula, right:Formula) :
        assert(conectivo in ['Y','O','>','='])
        self.conectivo = conectivo
        self.left = left
        self.right = right

def inorder_to_tree(cadena:str):
    conectivos = ['Y', 'O', '>', '=']
    if len(cadena) == 1:
        return Letra(cadena)
    elif cadena[0] == '-':
        return Negacion(inorder_to_tree(cadena[1:]))
    elif cadena[0] == "(":
        counter = 0 #Contador de parentesis
        for i in range(1, len(cadena)):
            if cadena[i] == "(":
                counter += 1
            elif cadena[i] == ")":
                counter -=1
            elif cadena[i] in conectivos and counter == 0:
                return Binario(cadena[i], inorder_to_tree(cadena[1:i]),inorder_to_tree(cadena[i + 1:-1]))
    else:
        raise Exception('¡Cadena inválida!')

In [ ]:
from Logica import *

In [ ]:
A = inorder_to_tree('--p')
I = {'p':True}
A.valor(I)

True

**Ejercicio 1:**

Implemente la función `chequear_equivalencia`.

Verifique su implementación para corroborar las siguientes equivalencias:

* $(p\to q)\equiv(\neg p\vee q)$
* $\neg(p\wedge q)\equiv (\neg p\vee\neg q)$
* $\neg(p\vee q)\equiv (\neg p\wedge\neg q)$
* $\bigl(p\vee(q\wedge r)\bigr)\equiv\bigl((p\vee q)\wedge(p\vee r)\bigr)$
* $\bigl(p\wedge(q\vee r)\bigr)\equiv\bigl((p\wedge q)\vee(p\wedge r)\bigr)$

Corrobore también que las siguientes NO son equivalencias:

* $(p\to q)\not\equiv(q\to p)$
* $\bigl(p\vee(q\wedge r)\bigr)\not\equiv\bigl((p\wedge q)\vee(p\wedge r)\bigr)$

In [ ]:
def chequear_equivalencia(variables, expr1, expr2):

    formula1 = inorder_to_tree(expr1)
    formula2 = inorder_to_tree(expr2)

    valores = list(product([True, False], repeat=len(variables)))
    for valor in valores:

        I = dict(zip(variables, valor))

        resultado1 = formula1.valor(I)
        resultado2 = formula2.valor(I)

        if resultado1 != resultado2:
            return False

    return True

variables = ['p', 'q','r']
print(chequear_equivalencia(variables, '(p>q)', '(-pOq)'))
print(chequear_equivalencia(variables, '-(pYq)', '(-pO-q)'))
print(chequear_equivalencia(variables, '-(pOq)', '(-pY-q)'))
print(chequear_equivalencia(variables, '(pO(qYr))', '((pOq)Y(pOr))'))
print(chequear_equivalencia(variables, '(pY(qOr))', '((pYq)O(pYr))'))
print(chequear_equivalencia(variables, '(p>q)', '(q>p)'))
print(chequear_equivalencia(variables, '(pO(qYr))', '((pYq)O(pYr))'))



True
True
True
True
True
False
False


In [ ]:
variables = ['p', 'q', 'r']
print(chequear_equivalencia(variables, '(p and not q) or (not p and q)', 'p != q'))

Exception: ignored

# Sustituciones <a class="anchor" id="sus"></a>

([Volver al inicio](#inicio))

Recordemos que el siguiente es el pseudocódigo de sustituir una subfórmula $A$ de $B$ por otra fórmula $A'$:

`
funcion sust(self, A, A'):
    Si A no está en self.subforms()
        retornar self
    Si no, si A es self
        retornar A'
    Si no, si self es de tipo Negacion
        retornar Negacion(self.subf.sust(A,A'))
    Si no, si self es de tipo Binario
        retornar Binario(self.conectivo,
                         self.left.sust(A,A'),
                         self.right.sust(A,A'))
`

**Ejercicio 2:**

Implemente el método sust(A,A').

**Nota:** No requiere implementar el método `subforms`, toda vez que este ya está implementado en la clase `Formula` de la librería `Logica`. Observe que dicho método `subforms` devuelve una lista de strings. Tenga esto en cuenta al momento de hacer las comparaciones requeridas por los dos primeros casos del método `sust`.


In [ ]:
def sust(self, A1, A2):
  if str(A1) not in self.subforms(): return self
  else:
    if str(A1) == str(self): return A2
    elif type(self) == Negacion: return Negacion(self.subforms.sust(A1,A2))
    elif type(self) == Binario: return Binario(self.conectivo, self.left.sust(A1,A2), self.right.sust(A1,A2))
setattr(Formula,"sust",sust)

In [ ]:
B = inorder_to_tree("-(pYq)")
B.subforms()

['q', 'p', '-(pYq)', '(pYq)']

Pruebe su código con el siguiente ejemplo. Debería obtener

`
Al reemplazar -(pYq) por s en (r>-(pYq))  se obtiene:
(r>s)
`

In [ ]:
B = inorder_to_tree("(r>-(pYq))")
A = inorder_to_tree('-(pYq)')
C = Letra('s')
D = B.sust(A,C)
print(f'Al reemplazar {str(A)} por {str(C)} en {str(B)}  se obtiene:\n{str(D)}')

Al reemplazar -(pYq) por s en (r>-(pYq))  se obtiene:
(r>s)


---

# Forma Normal Conjuntiva <a class="anchor" id="fnc"></a>

([Volver al inicio](#inicio))

Procedimiento para transformar una fórmula arbitraria $A$ en una fórmula $A'$ en forma normal conjuntiva, tal que $A\equiv A'$:

1. Eliminar `$\leftrightarrow$' y '$\to$'.
2. Eliminar dobles negaciones.
3. Si $\neg(B\wedge C)\in A.\mbox{subform}()$, reemplazarla por $\neg B\vee\neg C$.
4. Si $\neg(B\vee C)\in A.\mbox{subform}()$, reemplazarla por $\neg B\wedge\neg C$.
5. Eliminar dobles negaciones.
6. Si $B\vee (C\wedge D)\in A.\mbox{subform}()$, reemplazarla por $(B\vee C)\wedge (B\vee D)$.


In [ ]:
from copy import deepcopy

class Formula:

  def eliminar_imp(self):
    if type(self) == Letra:
        return self
    elif type(self) == Negacion:
        return Negacion(self.subf.eliminar_imp())
    elif type(self) == Binario:
        if self.conectivo == '>':
            return Binario('O', Negacion(self.left.eliminar_imp),
                           self.right.eliminar_imp
                           )
        else:
            return Binario(self.conectivo,
                       self.left.eliminar_imp(),
                       self.right.eliminar_imp()
                      )

setattr(Formula,"eliminar_imp",eliminar_imp)


def eliminar_doble_imp(self):
    if type(self) == Letra:
        return self
    elif type(self) == Negacion:
        return Negacion(self.subf.eliminar_doble_imp())
    elif type(self) == Binario:
        if self.conectivo == '=':
            return Binario('Y',
                           Binario('O',
                               Negacion(self.left.eliminar_doble_imp()),
                               self.right.eliminar_doble_imp(),
                              ),
                           Binario('O',
                               Negacion(self.right.eliminar_doble_imp()),
                               self.left.eliminar_doble_imp(),
                              ))
        else:
            return Binario(self.conectivo,
                       self.left.eliminar_doble_imp(),
                       self.right.eliminar_doble_imp()
                      )


def eliminar_doble_negacion(self):
    if type(self) == Letra:
        return self
    elif type(self) == Negacion:
        if type(self.subf) == Negacion:
            return Negacion(self.subf.subf.eliminar_doble_negacion())
        else:
            return Negacion(self.subf.eliminar_doble_negacion())
    elif type(self) == Binario:
        return Binario(self.conectivo,
                       self.left.eliminar_doble_negacion(),
                       self.right.eliminar_doble_negacion())



def cambiar_de_morgan_y(self):
    if type(self) == Letra:
        return self
    elif type(self) == Negacion:
        if type(self.subf) == Binario:
            if self.subf.conectivo == 'Y':
                return Binario('O',
                               Negacion(self.subf.left.cambiar_de_morgan_y()),
                               Negacion(self.subf.right.cambiar_de_morgan_y())
                              )
            else:
                return Negacion(self.subf.cambiar_de_morgan_y())
        else:
            return Negacion(self.subf.cambiar_de_morgan_y())
    elif type(self) == Binario:
        return Binario(self.conectivo,
                       self.left.cambiar_de_morgan_y(),
                       self.right.cambiar_de_morgan_y()
                      )

setattr(Formula,"cambiar_de_morgan_y",cambiar_de_morgan_y)

def cambiar_de_morgan_o(self):
    if type(self) == Letra:
        return self
    elif type(self) == Negacion:
        if type(self.subf) == Binario:
            if self.subf.conectivo == 'O':
                return Binario('Y',
                               Negacion(self.subf.left.cambiar_de_morgan_o()),
                               Negacion(self.subf.right.cambiar_de_morgan_o())
                              )
            else:
                return Negacion(self.subf.cambiar_de_morgan_o())
        else:
            return Negacion(self.subf.cambiar_de_morgan_o())
    elif type(self) == Binario:
        return Binario(self.conectivo,
                       self.left.cambiar_de_morgan_o(),
                       self.right.cambiar_de_morgan_o()
                      )

setattr(Formula,"cambiar_de_morgan_o",cambiar_de_morgan_o)

def distribuir_o_en_y(self):
    if type(self) == Letra:
        return self
    elif type(self) == Negacion:
        return Negacion(self.subf.distribuir_o_en_y())
    elif type(self) == Binario:
        if self.conectivo == 'O':
            if type(self.right) == Binario:
                if self.right.conectivo == 'Y': # B O (C Y D)
                    B = self.left.distribuir_o_en_y()
                    C = self.right.left.distribuir_o_en_y()
                    D = self.right.right.distribuir_o_en_y()
                    return Binario('Y',
                                   Binario('O', B, C),
                                   Binario('O', B, D)
                                  )
                else:
                    return Binario(self.conectivo,
                                   self.left.distribuir_o_en_y(),
                                   self.right.distribuir_o_en_y()
                                  )
            elif type(self.left) == Binario:
                if self.left.conectivo == 'Y': # (B Y C) O D
                    B = self.left.left.distribuir_o_en_y()
                    C = self.left.right.distribuir_o_en_y()
                    D = self.right.distribuir_o_en_y()
                    return Binario('Y',
                                   Binario('O', B, D),
                                   Binario('O', C, D)
                                  )
                else:
                    return Binario(self.conectivo,
                                   self.left.distribuir_o_en_y(),
                                   self.right.distribuir_o_en_y()
                                  )
        else:
            return Binario(self.conectivo,
                           self.left.distribuir_o_en_y(),
                           self.right.distribuir_o_en_y()
                          )


setattr(Formula,"distribuir_o_en_y",distribuir_o_en_y)


In [ ]:
A = inorder_to_tree('--(-(r>q)=(--pYq))')
B = A.eliminar_doble_imp()
print(f'Al eliminar las dobles implicaciones de \n{str(A)}\nse obtiene:\n{str(B)}')
C = B.eliminar_doble_negacion()
print(f'Al eliminar las dobles negaciones se obtiene:\n{str(C)}')
D = C.cambiar_de_morgan_y()
print(f'Al reemplazar -(AYB) por -AO-B se obtiene:\n{str(D)}')


AttributeError: ignored

**Ejercicio 3:**

Complete todos los métodos de modificación anteriores. Al probar su código con la siguiente celda, la respuesta debe ser:

`
Fórmula inicial:
((pOq)>(rY-s))
Al eliminar las implicaciones se obtiene:
(-(pOq)O(rY-s))
Al reemplazar -(AOB) por -AY-B se obtiene:
((-pY-q)O(rY-s))
Al distribuir O en Y se obtiene:
(((-pY-q)Or)Y((-pY-q)O-s))
Al distribuir O en Y se obtiene:
(((-pOr)Y(-qOr))Y((-pO-s)Y(-qO-s)))
`

In [ ]:
A = inorder_to_tree("((pOq)>(rY-s))")
print(f'Fórmula inicial:\n{str(A)}')
B = A.eliminar_imp()
print(f'Al eliminar las implicaciones se obtiene:\n{str(B)}')
C = B.cambiar_de_morgan_o()
print(f'Al reemplazar -(AOB) por -AY-B se obtiene:\n{str(C)}')
D = C.distribuir_o_en_y()
print(f'Al distribuir O en Y se obtiene:\n{str(D)}')
E = D.distribuir_o_en_y()
print(f'Al distribuir O en Y se obtiene:\n{str(E)}')



Fórmula inicial:
((pOq)>(rY-s))


AttributeError: ignored

---